Evaluate the models

# Stochastic Adversarial Training (StochAT)

### SoTA

vanila SGD: 
MNIST - 99%+ (most cnns), CIFAR10 - 93%+ (resnet18), 96%+ (wideresnet) 

MNIST:

adversarial attacks: 
l-inf @ eps = 80/255 @20 steps: TRADES - 96.07% - (4 layer cnn), MART 96.4%, MMA 95.5%, PGD - 96.01% - (4 layer cnn)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MMA, PGD

CIFAR10:

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
TRADES 53-56% - (WRN-34-10), MART 57-58% (WRN-34-10), MMA 47%, PGD 48% - (WRN-32-10)// 49% - (WRN-34-10), Std - 0.03%
https://openreview.net/pdf?id=rklOg6EFwS (Table 4)

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
[ResNet10] TRADES 45.4%, MART 46.6%, MMA 37.26%, PGD 42.27%, Std 0.14%

Benign accuracies: TRADES 84.92%, MART 83.62%, MMA 84.36, PGD 87.14%, Std 95.8% [wideresnet]
https://openreview.net/pdf?id=Ms9zjhVB5R (Table 1)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MART, MMA, PGD

TBD: CWinf attacks

## Pretrained models for comparison

download pretrained models and place in ../trainedmodels/MNIST or ../trainedmodels/CIFAR10 respectively

### TRADES :
https://github.com/yaodongyu/TRADES (MNIST: small cnn, CIFAR10: WideResNet34)
### MMA : 
https://github.com/BorealisAI/mma_training (MNIST: lenet5, CIFAR10: WideResNet28)
### MART :
 https://github.com/YisenWang/MART (CIFAR10: ResNet18 and WideResNet34)

## IMPORT LIBRARIES

In [49]:
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from multiprocessing import cpu_count
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import olympic
from typing import Union, Callable, Tuple
import sys
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
from functional import boundary, iterated_fgsm, local_search, pgd, entropySmoothing
from ESGD_utils import *
import pickle
import time
import torch.backends.cudnn as cudnn
import argparse, math, random
import ESGD_optim
from trades import trades_loss

In [2]:
from torch.autograd import Variable

In [3]:
from torch.optim.lr_scheduler import StepLR

In [4]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [5]:
from utils import project

In [6]:
DEVICE

'cuda'

# LOAD DATA

In [7]:
#place data folders outside working directory

In [8]:
kwargs = {'num_workers': 4, 'pin_memory': True}

In [44]:
args = {}
args['test_batch_size'] = 128
args['no_cuda'] = False
args['epsilon'] = 0.3
args['num_steps'] = 20
args['step_size'] = 0.02
args['random'] =True,
args['white_box_attack']=True
args['log_interval'] = 100
args['beta'] = 1
args['seed'] = 1
args['lr'] = 0.03
args['momentum'] = 0.9
args['batch_size'] = 128
args['save_freq'] = 5
args['attack'] = 'l_inf'
args['norm'] = 'inf'
args['epochs'] = 30

In [26]:
dataset = 'MNIST' # [MNIST, CIFAR10]
if dataset == 'MNIST':
    transform = transforms.Compose([
    transforms.ToTensor()])
    train = datasets.MNIST('../../data/', train=True, transform=transform, download=True)
    val = datasets.MNIST('../../data/', train=False, transform=transform, download=True)
elif dataset == 'CIFAR10':
    # setup data loader
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])
    transform_test = transforms.Compose([
        transforms.ToTensor(),
    ])

    train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform_train, download=True)
    val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform_test, download=True)
    
train_loader = DataLoader(train, batch_size=args['batch_size'], shuffle=True, **kwargs)
val_loader = DataLoader(val, batch_size=args['test_batch_size'], shuffle=False, **kwargs)

# INITIALIZE NETWORK

In [27]:
if dataset=='MNIST':
    #from net_mnist import Net
    #NetName = 'net_mnist'
    from small_cnn import SmallCNN   
    Net = SmallCNN
    NetName = 'SmallCNN'

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 1024)
        x = self.fc1(x)
        return x#F.log_softmax(x, dim=1)

In [28]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    Net = WideResNet
    NetName = 'WideResNet'

In [29]:
Net

small_cnn.SmallCNN

# RANDOM SEED 

In [30]:
seed = args['seed']
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [31]:
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy

In [32]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [45]:
def _pgd_whitebox(model,
                  X,
                  y,
                  epsilon=args['epsilon'],
                  norm=args['norm'],
                  num_steps=args['num_steps'],
                  step_size=args['step_size']):
    out = model(X)
    err = (out.data.max(1)[1] != y.data).float().sum()
    X_pgd = Variable(X.data, requires_grad=True)
    if args['random']:
        random_noise = torch.FloatTensor(*X_pgd.shape).uniform_(-epsilon, epsilon).to(DEVICE)
        X_pgd = Variable(X_pgd.data + random_noise, requires_grad=True)

    for _ in range(num_steps):
        opt = optim.SGD([X_pgd], lr=1e-3)
        opt.zero_grad()

        with torch.enable_grad():
            loss = nn.CrossEntropyLoss()(model(X_pgd), y)
        loss.backward()
        if norm=='inf':
            eta = step_size * X_pgd.grad.data.sign()
            X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
            eta = torch.clamp(X_pgd.data - X.data, -epsilon, epsilon)
            X_pgd = Variable(X.data + eta, requires_grad=True)
        elif norm==2:
            #print('l2 attack')
            eta = step_size * X_pgd.grad.data / X_pgd.grad.view(X_pgd.shape[0], -1).norm(2, dim=-1)\
                    .view(-1, 1, 1, 1)
            X_pgd = Variable(X_pgd.data + eta, requires_grad=True)
            X_pgd = project(X, X_pgd, norm, epsilon)            
        X_pgd = Variable(torch.clamp(X_pgd, 0, 1.0), requires_grad=True)
        #print('distance of attack:',torch.norm(X_pgd-X)/np.sqrt(128))
    err_pgd = (model(X_pgd).data.max(1)[1] != y.data).float().sum()
    #print('err pgd (white-box): ', err_pgd)
    return err, err_pgd

def eval_adv_test_whitebox(model, device, test_loader):
    """
    evaluate model by white-box attack
    """
    model.eval()
    robust_err_total = 0
    natural_err_total = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        # pgd attack
        X, y = Variable(data, requires_grad=True), Variable(target)
        err_natural, err_robust = _pgd_whitebox(model, X, y)
        robust_err_total += err_robust
        natural_err_total += err_natural
    rob = 100-robust_err_total.item()/100    
    print('natural_acc_total: ', 100-natural_err_total.item()/100)
    print('robust_acc_total: ', rob)
    return rob

In [34]:
if not os.path.exists('../trainedmodels'):
    os.makedirs('../trainedmodels')

In [35]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [36]:
def train_adversarial(method,model, device, train_loader, optimizer, epoch,adversary,L,step,eps,norm):
    totalcorrect = 0
    batches = len(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        #print('batch:{:d}/{:d}'.format(batch_idx,batches))
        data, target = data.to(device), target.to(device)        
        sgd_loss = nn.CrossEntropyLoss()
        # calculate robust loss per batch
        loss, correct = method(model,optimizer,sgd_loss,data,target,epoch,adversary,L,step,eps,norm)
        totalcorrect += correct
    print('robust train accuracy:',100*totalcorrect/len(train_loader.dataset))   

# TRAIN MODEL USING SAT

In [46]:
def adversarial_training(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm):
    """Performs a single update against a specified adversary"""
    model.train()
    
    # Adversial perturbation
    x_adv = adversary(model, x, y, loss_fn, k=k, step=step, eps=eps, norm=norm, random=True)
    
    optimiser.zero_grad()
    y_pred = model(x_adv)
    pred = y_pred.max(1, keepdim=True)[1]
    correct = pred.eq(y.view_as(pred)).sum().item()
    loss = loss_fn(y_pred, y)
    loss.backward()
    optimiser.step()

    return loss, correct

In [38]:
def adversarial_training_entropy(model, optimiser, loss_fn, x, y, epoch, adversary, L, step, eps, norm):
    """Performs a single update against a specified adversary"""
    model.train()
    
    # Adversial perturbation
    alpha=0.9
    loss = 0
    
    if norm==2:
        gamma = 1e-2
    elif norm=='inf':
        gamma = 0
        
    for l in range(L):     
        
        if l==0: ## initialize using random perturbation of true x, run for one epoch
            k=1
            random=True
            xp = None
            projector=False
        elif l>0 and l<L-1: ## initialize with previous iterate of adversarial perturbation, run one epoch
            k=1
            random=False
            xp=x_adv
            projector = False
        elif l == L-1: ## initialize with previous iterate, run one epoch, project to epsilon ball
            k=1
            random=False
            xp = x_adv
            if norm==2:
                projector=False
            elif norm=='inf':    
                projector=True
            
        x_adv = adversary(model, x, y, loss_fn, xp=xp, step=step, eps=eps, norm=norm, random=random, ep=1e-4,projector=projector,gamma=gamma)
        
        optimiser.zero_grad()
        y_pred = model(x_adv)
        pred = y_pred.max(1, keepdim=True)[1]
        correct = pred.eq(y.view_as(pred)).sum().item()
        loss = (1-alpha)*loss + alpha*loss_fn(y_pred, y)
        
    loss.backward()
    optimiser.step()
    return loss, correct

In [39]:
model_dir = '../SmallCNN_MNIST'

In [40]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [47]:
def adjust_learning_rate(optimizer, epoch,lr_init):
    """decrease the learning rate"""
    lr = lr_init
    if epoch >= 15:   
        lr = lr_init * 0.5
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [48]:
## initialize model
model_ATENT = Net().to(DEVICE)
## training params
epochs = args['epochs']  
lr_init = 1*args['lr']
optimizer = optim.SGD(model_ATENT.parameters(), lr=lr_init, momentum=0.9)
## train model

for epoch in range(1, epochs+1):
    print('Epoch:',epoch)
    
    # adjust learning rate for SGD
    adjust_learning_rate(optimizer, epoch,lr_init)
    
    # adversarial training
    train_adversarial(adversarial_training_entropy,model_ATENT, DEVICE, train_loader, optimizer, epoch,adversary=entropySmoothing,L=2*args['num_steps'],step=args['step_size'],eps=args['epsilon'],norm=args['norm'])

    # evaluation on natural and adversarial examples
    print('================================================================')
    eval_train(model_ATENT, DEVICE, train_loader)
    rob = eval_adv_test_whitebox(model_ATENT, DEVICE, val_loader)            
    print('================================================================')

    # save checkpoint
    if (epoch-1) % 1 == 0:
        torch.save(model_ATENT.state_dict(),
                   os.path.join(model_dir, 'model-nn-epoch{}-robacc{}.pt'.format(epoch,int(np.round(rob)))))

## save model
modelname = '../trainedmodels/'+dataset+'/'+NetName+'_ATENT_'+args['attack']+'_ep'+str(epochs)+'_lr'+str(lr_init)+'.pt'
#torch.save(model_ATENT,modelname)

Epoch: 1
robust train accuracy: 41.815


/usr/lib/python3/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Training: Average loss: 0.3015, Accuracy: 57565/60000 (96%)
natural_acc_total:  95.99
robust_acc_total:  73.28999999999999
Epoch: 2
robust train accuracy: 88.16
Training: Average loss: 0.1630, Accuracy: 58774/60000 (98%)
natural_acc_total:  98.01
robust_acc_total:  85.98
Epoch: 3
robust train accuracy: 93.90333333333334
Training: Average loss: 0.0644, Accuracy: 59178/60000 (99%)
natural_acc_total:  98.82
robust_acc_total:  91.6
Epoch: 4
robust train accuracy: 95.51333333333334
Training: Average loss: 0.0476, Accuracy: 59315/60000 (99%)
natural_acc_total:  98.81
robust_acc_total:  93.35
Epoch: 5
robust train accuracy: 96.445
Training: Average loss: 0.0357, Accuracy: 59466/60000 (99%)
natural_acc_total:  98.9
robust_acc_total:  94.27
Epoch: 6
robust train accuracy: 96.90333333333334
Training: Average loss: 0.0379, Accuracy: 59482/60000 (99%)
natural_acc_total:  99.07
robust_acc_total:  94.88
Epoch: 7
robust train accuracy: 97.22166666666666
Training: Average loss: 0.0306, Accuracy: 59592

robust train accuracy: 99.255
Training: Average loss: 0.0047, Accuracy: 59944/60000 (100%)
natural_acc_total:  99.42
robust_acc_total:  95.93


In [ ]:
def train_trades(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=args['step_size'],
                           epsilon=args['epsilon'],
                           perturb_steps=args['num_steps'],
                           beta=args['beta'])

        loss.backward()
        optimizer.step()

In [ ]:
## initialize model
model_TRADES = Net().to(DEVICE)
## training params
lr_init = args['lr']
optimizer = optim.SGD(model_TRADES.parameters(), lr=lr_init, momentum=0.9)
epochs = args['epochs']
## train model

for epoch in range(1, epochs + 1):
    print('Epoch:',epoch)
    # adjust learning rate for SGD
    adjust_learning_rate(optimizer, epoch, lr_init = lr_init)

    # adversarial training
    train_trades(args, model_TRADES, DEVICE, train_loader, optimizer, epoch)

    # evaluation on natural examples
    if epoch % 1 == 0:
        print('================================================================')
        eval_train(model_TRADES, DEVICE, train_loader)
        eval_adv_test_whitebox(model_TRADES, DEVICE, val_loader)
        print('================================================================')

## save model
modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADES_'+args['attack']+'_ep'+str(epochs)+'_lr'+str(lr_init)+'.pt'
#torch.save(model_TRADES,modelname)